In [2]:
%pip install -q torch


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[3.9046e-01, 4.8469e-01, 7.9640e-01],
        [3.7582e-01, 3.6753e-01, 1.2290e-01],
        [6.1870e-05, 7.5596e-03, 9.6538e-01],
        [6.9563e-01, 2.0441e-01, 2.2054e-01],
        [3.5104e-01, 2.5844e-01, 7.8544e-01]])


In [7]:
torch.cuda.is_available()
torch.__version__

'2.4.0'

In [8]:
%pip install "unsloth[colab-new] @git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /private/var/folders/wd/k6f6pm051cq8gm__9_vyfzfm0000gp/T/pip-install-hugzyjy_/unsloth_025cf7c9f1db4173a091d3929ac2f79f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /private/var/folders/wd/k6f6pm051cq8gm__9_vyfzfm0000gp/T/pip-install-hugzyjy_/unsloth_025cf7c9f1db4173a091d3929ac2f79f
  Resolved https://github.com/unslothai/unsloth.git to commit 4e570be9ae4ced8cdc64e498125708e34942befc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 660.1 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 606.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 1.3 MB/s

In [9]:
%pip install --no-deps xformers trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.0 MB/s eta 0:00:0031m10.4 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [232 lines of output]
      running bdist_wheel
      /Users/jimmy.liao/.pyenv/versions/3.10.13/envs/lab_gemini/lib/python3.10/site-packages/torch/utils/cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      r